## Rational Function Optimization

Rational Function Optimization (RFO) is the method of choice for minimizations. This tutorial will walk through the basic theory of RFO and show a sample calculation.  The method was introduced for geometry optimizations by A. Banerjee, N. Adams, J. Simons, and R. Shepard in J. Phys. Chem. 89, 52(1985).

In the Newton-Raphson method, the potential energy surface is approximated by the truncated Taylor expansion in internal coordinates $q$ and gradient $g$ (where vectors are interpreted as columns).

$$ \epsilon = E(q) - E_0 = g^T \Delta q + \frac{1}{2}(\Delta q)^T \mathbf{H} \Delta q $$

An extension is to express the potential via a [2/2] Pade approximation, where __S__ is the scaling matrix.  If __S__ were zero, then the harmonic approximation is obtained.

$$ \epsilon = \frac{g^T\Delta q + \frac{1}{2} (\Delta q)^T \textbf{H} \Delta q}{1 + \Delta q^T \textbf{S} \Delta q} $$


The relative energy expression can be rewritten in the form of $N+1$ dimensional vectors, where $N$ is the number of coordinates.

$$ \epsilon = \frac{ \frac{1}{2}\begin{pmatrix} \Delta{q^T} & 1\end{pmatrix}\begin{pmatrix} {\textbf H} & g \\ g^T & 0  \end{pmatrix}\begin{pmatrix}\Delta{q} \\ 1\end{pmatrix} }{ \begin{pmatrix} \Delta q^T & 1 \end{pmatrix} \begin{pmatrix} \textbf S & 0  \\ 0 & 1 \end{pmatrix} \begin{pmatrix} \Delta q \\ 1 \end{pmatrix}}$$

since the right-hand side is
\begin{align}
  &= \frac{ \frac{1}{2}\begin{pmatrix} \Delta{q^T} & 1\end{pmatrix} \begin{pmatrix} \textbf{H} \Delta q  + g \\ g^T \Delta q + 0 \end{pmatrix}}{\begin{pmatrix} \Delta q^T & 1 \end{pmatrix} \begin{pmatrix} \textbf S \Delta q + 0 \\ 0 + 1 \end{pmatrix}} \\
  \\
 &= \frac { \frac{1}{2} \Delta q^T \textbf H \Delta q + \frac{1}{2} \Delta q^T g + \frac{1}{2} g^T \Delta q} { \Delta q^T\textbf S \Delta q + 1}
 \\
\end{align}
which is equivalent to the expression above.

Making the stationary point assumption that  
$$\frac{\partial \epsilon}{\partial q } = 0 $$
we can derive the expression for the step.

\begin{align}
\frac{\partial \epsilon}{\partial q} &= \frac {g + \mathbf{H} \Delta q}{1 + \Delta q^T \textbf S \Delta q} - \frac{ g \Delta q^T + \frac{1}{2} \Delta q^T \textbf H \Delta q}{ 1 + \Delta q^T \textbf S \Delta q} \Big( \frac{ 2 \textbf S \Delta q}{1 + \Delta q^T \textbf S \Delta q}\Big) \\
\\
\frac{\partial \epsilon}{\partial q} &= \frac {g + \textbf{H}\Delta q}{1 + \Delta q^T \textbf S \Delta q} - \epsilon \Big( \frac{ 2 \textbf S \Delta q}{1 + \Delta q^T \textbf S \Delta q} \Big) \\
\\
0 &= \frac{ g + \textbf{H} \Delta q - 2 \epsilon \textbf S \Delta q}{1 + \Delta q^T \textbf S \Delta q}\\
\\
0 &=  g + \textbf{H} \Delta q - 2 \epsilon \textbf S \Delta q \\
\\
g + H\Delta q &= 2 \epsilon \textbf S \Delta q = \lambda \textbf S \Delta q\\
\end{align}

where $\lambda$ is defined as $2\epsilon$.  It can be shown that 
$$\lambda = g^T \Delta q$$

which allows the stationarity condition to be written as follows.

\begin{align}
\begin{pmatrix} \textbf H \Delta q \\ g^T \Delta q \end{pmatrix} + \begin{pmatrix} g \\ 0 \end{pmatrix} &= \begin{pmatrix} \lambda \textbf S \Delta q \\ \lambda \end{pmatrix} \\
\textrm{or} \\
\begin{pmatrix} \textbf H & g \\ g^T & 0 \end{pmatrix} \begin{pmatrix} \Delta q \\ 1 \end{pmatrix} &= \lambda \begin{pmatrix} {\textbf{S} \Delta q} \\ 1 \end{pmatrix} \\
\end{align}

The matrix __S__ is usually taken to be the identify matrix.  The result is an eigenvalue equation with $\lambda$ as the eigenvalue!

$$
\begin{pmatrix} \textbf H & g \\ g^T & 0 \end{pmatrix}
\begin{pmatrix} \Delta q \\ 1 \end{pmatrix} = \lambda \begin{pmatrix} { \Delta q} \\ 1 \end{pmatrix}
$$


The $N+1$ dimensional matrix on the left is called the "RFO matrix".  We build it from our Hessian and our gradient in internal coordinates.  Then the eigenvectors and eigenvalues of this matrix are determined.  For minimum-energy searches, we usually choose the eigenvector with the lowest value of $\lambda$.  This value is hopefully negative, and is 2 times the  energy change anticipated from the step.  This eigenvector is intermediate-normalized by scaling the last element to 1.  The rest of the vector is the desired RFO step, or displacement in internal coordinates.

Occasionally, this eigenvector has a very small final element, and cannot be intermediate normalized.  This comes about, for example, when the step breaks molecular symmetry and the projected energy change may be zero.  Also, the algorithm is numerically problematic if the gradients are very small.

In practice, RFO performs better than ordinary Newton-Raphson steps if the energy surface being explored is not very harmonic, or if the optimization is begun far from a minimum.  However, if the potential surface is nearly flat (e.g., methane dimer), then RFO like ordinary N-R, will perform poorly.

In [ ]:
import psi4
from optking import optParams as op
from optking import printTools
printTools.printInit(printTools.cleanPrint)
mol = psi4.molecule ("""
  H     0.0000000000   0.9803530335  -0.8498671785
  O     0.0000000000   0.6988545188   0.0536419016
  O     0.0000000000  -0.6988545188   0.0536419016
  H     0.0000000000  -0.9803530335  -0.8498671785
"""")

The Partioned-RFO method is an extension of the RFO method, and may be used to seek non-minima stationary points such as transition states.  In the P-RFO method, there are two RFO matrices.  We maximize along 1 or more degrees of freedom while minimizing along the others.

An extended scheme to limit the RFO step size may be found in
Besalu and Bofill, Theor. Chem. Acc., 1999, 100:265-274
E. Besalu and J.M. Bofill, Theor. Chem. Acc  100 (1998) 265.